In [1]:
"""
This notebook finds black/redacted areas and segments them (green is default), such that digital redaction may be performed downstream.
"""

'\nThis notebook finds black/redacted areas and segments them (green is default), such that digital redaction may be performed downstream.\n'

In [2]:
from PIL import Image
from scipy.ndimage import gaussian_filter
from skimage.measure import label, regionprops
from skimage.segmentation import expand_labels
from typing import Tuple
from IPython.display import display
import plotly.graph_objects as go
import tqdm
import os
import matplotlib.pyplot as plt
import numpy as np

In [3]:
ECG_FOLDER = '../../data/ecg_ahus_phone'
SAVE_FOLDER = '../../data/redacted_ecg_ahus_phone'

ECG_FOLDER = '/home/datasets/ecg-digitization/digital_redaction/agnar_phone_all'
SAVE_FOLDER = '/home/datasets/ecg-digitization/digital_redaction/agnar_phone_all_redacted'
ECG_FOLDER = '/home/datasets/ecg-digitization/digital_redaction/elias_phone_all'
SAVE_FOLDER = '/home/datasets/ecg-digitization/digital_redaction/elias_phone_all_redacted_2'

elias_params =  [1.5e-2, 1.4e-2, 1.3e-2, 1.2e-2, 1.2e-2, 1e-2, 9e-3, 8e-3, 7e-3, 6e-3, 5e-3, 4e-3, 3e-3, 2e-3, 1e-3]
agnar_params =  [1.3e-2, 1.2e-2, 1.1e-2, 1e-2, 9e-3, 8e-3, 7e-3, 6e-3, 5e-3, 4e-3, 3e-3, 2e-3, 1e-3]
REDACTED_AREA_PROPORTIONS = elias_params

DISPLAY = False

In [ ]:
def redact_image(
        image_path: str,
        min_redacted_area_proportion: float = 1e-2,
        redacted_factor: float = 1.2,
        expand_redact_pixels: int = 10,
        display: bool = False
    ) -> Tuple[np.ndarray, np.ndarray]:


    img = Image.open(image_path)
    img_gray = img.convert('L')
    img_np = np.array(img_gray)

    redacted_treshold_grayscale = np.percentile(img_np.flatten(), min_redacted_area_proportion*100)

    redacted_treshold_grayscale = redacted_treshold_grayscale * redacted_factor
    print(redacted_treshold_grayscale)
    print(f'Redacted threshold grayscale: {redacted_treshold_grayscale}')


    if display:
        plt.imshow(img_np)

    img_blurred = gaussian_filter(img_np, sigma=3)
    redacted = img_blurred < redacted_treshold_grayscale

    if display:
        plt.figure()
        plt.imshow(redacted, cmap='gray')
        plt.show()


    labeled = label(redacted)
    regions = regionprops(labeled)

    regions_filtered = []
    ids = []

    min_area_proportion = 1.3e-4
    min_area_perimeter_coefficient = 10
    for region in regions:
        if region.area / img_np.size < min_area_proportion:
            continue
        if region.area / region.perimeter < min_area_perimeter_coefficient:
            continue

        regions_filtered.append(region)
        ids.append(region.label)


    areas = [region.area for region in regions_filtered]
    perimeters = [region.perimeter for region in regions_filtered]

    if display:
        plt.plot(areas, perimeters, 'o')
        plt.xlabel('Area')
        plt.ylabel('Perimeter')
        plt.title('Area vs Perimeter of Regions')
        plt.grid()

    regions_after_filtering = np.isin(labeled, ids)

    if display:
        s = int(np.sqrt(min_area_proportion * img_np.size)//2)
        print(f'Showing {s} pixels around the redacted regions')
        regions_after_filtering[regions_after_filtering.shape[0]//2-s:regions_after_filtering.shape[0]//2+s, regions_after_filtering.shape[1]//2-s:regions_after_filtering.shape[1]//2+s] = 255
        plt.figure()
        plt.imshow(regions_after_filtering, cmap='gray')

    regions_after_filtering = expand_labels(regions_after_filtering, distance=expand_redact_pixels)

    if display:
        plt.figure()
        plt.imshow(regions_after_filtering, cmap='gray')

    img_redacted = np.array(img)
    img_redacted[regions_after_filtering > 0] = np.array([0, 255, 0])

    if display:
        plt.figure()
        plt.hist(img_np.flatten(), bins=256, color='green', alpha=0.7)
        plt.title('Histogram of Redacted Image Pixel Values')
        plt.xlabel('Pixel Value')
        plt.ylabel('Frequency')
        plt.grid()


    return img_redacted, regions_after_filtering


names = []
sizes = []


scatter = go.Scatter(
    x=[], y=[], mode='markers+text',
    text=[], textposition='top center',
    marker=dict(size=10)
)

fig = go.FigureWidget(data=[scatter])
display(fig)

image_names = tqdm.tqdm(os.listdir(ECG_FOLDER), desc='Redacting images', unit='image')

for image_name in image_names:
    path = os.path.join(ECG_FOLDER, image_name)
    save_path = os.path.join(SAVE_FOLDER, image_name)

    redacted_image = None
    regions_after_filtering = None

    for r in REDACTED_AREA_PROPORTIONS:
        curr_redacted_image, curr_regions_after_filtering = redact_image(path, min_redacted_area_proportion=r, display=DISPLAY)
        max_area = max([a.area for a in regionprops(label(curr_regions_after_filtering > 0))])

        if max_area / curr_regions_after_filtering.size < 1.4e-2:
            redacted_image = curr_redacted_image
            regions_after_filtering = curr_regions_after_filtering
            break



    proportion_redacted = np.sum(regions_after_filtering > 0) / regions_after_filtering.size
    names.append(image_name)
    sizes.append(proportion_redacted)

    if proportion_redacted < 0.002:
        print(f'Low redaction proportion for {image_name}: {proportion_redacted:.4f}')

    Image.fromarray(redacted_image).save(save_path)

    with fig.batch_update():
        fig.data[0].x = np.zeros_like(sizes)
        fig.data[0].y = sizes
        fig.data[0].text = names



FigureWidget({
    'data': [{'marker': {'size': 10},
              'mode': 'markers+text',
              'text': [],
              'textposition': 'top center',
              'type': 'scatter',
              'uid': 'bcb2c326-55a5-4ed4-b056-f332033b48de',
              'x': [],
              'y': []}],
    'layout': {'template': '...'}
})

Redacting images:   0%|          | 0/558 [00:00<?, ?image/s]

IMG20250616153948.jpg
81.6
Redacted threshold grayscale: 81.6


Redacting images:   0%|          | 1/558 [00:01<10:31,  1.13s/image]

IMG20250617082343.jpg
74.39999999999999
Redacted threshold grayscale: 74.39999999999999


...

Redacting images: 100%|█████████▉| 556/558 [14:58<00:03,  1.78s/image]

IMG20250617082216.jpg
86.39999999999999
Redacted threshold grayscale: 86.39999999999999


Redacting images: 100%|█████████▉| 557/558 [15:00<00:01,  1.80s/image]

IMG20250617081815.jpg
116.39999999999999
Redacted threshold grayscale: 116.39999999999999


Redacting images: 100%|██████████| 558/558 [15:01<00:00,  1.62s/image]
